<div align="center">
  <h1 style="color: orange;">ReAct</h1>
</div>

- <span style="font-size: 20px;"><b>ReAct: Synergizing Reasoning And Acting In Language Models</b></span>  
  [https://arxiv.org/pdf/2210.03629](https://arxiv.org/pdf/2210.03629)
- <span style="font-size: 20px;"><b>Reason - Act - Observe - Iterate loop</b></span>
- <span style="font-size: 20px;"><b>Tool calling support</b></span>
- <span style="font-size: 20px;"><b>Prebuilt Agent Limitations</b></span>

<img src="./graph.png" alt="ReAct Flowchart" style="float: right; width: 300px;">

In [ ]:
import json
from typing import Type

from dotenv import load_dotenv
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel

load_dotenv()

# Precoded tools (not dynamic)
toolkit = GitHubToolkit.from_github_api_wrapper(GitHubAPIWrapper())
tools = [setattr(tool, "name", tool.mode) or tool for tool in toolkit.get_tools()]

llm = ChatOpenAI(model="gpt-4o", temperature=0)

def ai(query: str, response_format: Type[BaseModel]):
    agent_executor = create_react_agent(llm, tools, response_format=response_format)
    events = agent_executor.stream(
        {"messages": [("user", query)]},
        stream_mode="values",
    )
    event = {}
    for event in events:
        event["messages"][-1].pretty_print()

    structured_response = event.get("structured_response")
    data = {"data": [structured_response.model_dump()]}
    print(data)
    return json.dumps(data)


In [ ]:
from dotenv import load_dotenv
from flask import Flask
from flask_cors import CORS
from pydantic import BaseModel

# Load environment variables from .env
load_dotenv()

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "http://localhost:3000"}})


class Author(BaseModel):
    """Respond to the user in this format."""
    name: str
    issue_count: int
    comment_count: int


@app.route('/data', methods=['GET'])
def get_data():
    query = ("For the most prolific author of github issues, give me the number of issues they have created. "
             "Given the author's issues, I want to know the number of comments that same author has made." )

    # We are trying to get rid of engineering as much as possible, but Author and tools implementation is still in the way
    return ai(query, response_format=Author)
    

## Goals
- ~~Write natural language only~~
- ~~Unstructured data~~
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~

In [ ]:
app.run(debug=False, port=5001, use_reloader=False)